In [ ]:
%matplotlib notebook
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

### Ejemplo de inferencia paramétrica

- Queremos estimar los parámetros $\mu$ y $\sigma^2$ de la población
- A partir de las observaciones construimos una estadístico muestral: $\bar X = \sum_i X_i/N$
- Si **asumimos** que la población se distribuye normal entonces: $\bar X = \mathcal{N}(\mu, \sigma^2/N)$, es decir que 
    - En promedio la media muestral equivale a la media de la población
    - A mayor número de muestras menor es la varianza de nuestro estimador
- Para tomar decisiones basados en $\bar X$ usamos un estadístico de test: $t = \frac{\bar X - \mu}{\sqrt{\hat \sigma^2/N}}$ (o estadístico Z si $\sigma$ es conocido)
- Dado que conocemos la distribución de $\bar X$ podemos calcular la distribución de $t$ bajo la hipótesis nula (e.g. $\mu=0$) y calcular el p-value 

> p-value: pbb de observar un valor más grande que t si la hipótesis nula fuera cierta

### El análisis descansa en los supuestos sobre la distribución de la población

¿Qué ocurre en el ejemplo anterior si el supuesto sobre la distribución población estaba equivocado?

La distribución muestral será erronea, la distribución nula será erronea y el p-value será erroneo

### Horror ¿Qué hago?

1. (Mantén la calma)
1. En la práctica los tests parámetricos funcionan con "ligeras" desviaciones de los supuestos
1. Podemos comprobar los supuestos antes de hacer el análisis
1. Si los supuestos no se cumplen podemos usar **inferencia no paramétrica**

# Evaluando normalidad

La normalidad es un supuesto usado en muchos procedimientos parámetricos: t-test, ANOVA, regresion lineal

Tengamos en cuanta las características principales de la distribución normal
- Unimodal con media igual a su moda
- Simétrica en torno a la media
- Concentrada en torno a la media: ~68% a $\pm \sigma$, ~95% a $\pm 2\sigma$, ...

Si tenemos suficientes muestras podriamos observar el histograma y comprobar visualmente si esto se cumple

In [ ]:
data1 = scipy.stats.norm(loc=4, scale=2).rvs(1000)
data2 = scipy.stats.gamma(a=2, scale=1).rvs(1000)
data3 = scipy.stats.uniform(loc=2, scale=1).rvs(1000)

fig, ax = plt.subplots(1, 3, figsize=(7, 2.5), tight_layout=True)
for i, data in enumerate([data1, data2, data3]):
    #data = (data - np.mean(data))/np.std(data)
    ax[i].hist(data, bins=10, density=True)
    ax[i].set_xlabel('data'+str(i))

### Momentos estadísticos

En el caso particular de la normal sabemos que el tercer momento (simetría) debiese ser cero

Podemos calcular la simetría usando [`scipy.stats.skew`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.skew.html), para el caso particular de la normal puede ser util [`scipy.stats.skewtest`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.skewtest.html)

In [ ]:
for i, data in enumerate([data1, data2, data3]):
    display(scipy.stats.skew(data))

El cuarto momento (kurtosis) también tiene un valor particular en el caso de la distribución normal

La función [`scipy.stats.kurtosis`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kurtosis.html) tiene dos definiciones de Kurtosis

La definición por defecto (Fisher) nos da cero en el caso de que la distribución sea normal

Puede ser también util [`scipy.stats.kurtosistest`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kurtosistest.html)

In [ ]:
for i, data in enumerate([data1, data2, data3]):
    display(scipy.stats.kurtosis(data))

### Probability plots

Otra opción gráfica son los *probability plots* (Chambers et al., 1983) 

Se calculan los cuantiles (qq) o percentiles (pp) de dos muestras o de una muestra y una distribución teórica

Luego se grafican. Mientras más se parezca el resultado a una linea recta más similares son las distribuciones

Para mayores detalles teóricos: https://itl.nist.gov/div898/handbook/eda/section3/probplot.htm

Podemos usar `scipy.stats.probplot` para comparar una muestra con una distribución teórica 

- En este caso la distribución teórica será la normal estándar
- Dado que la prueba es basada en rangos no necesitamos entregar parámetros de localización (media) o escala (varianza)
- Cualquier parámetro de otro tipo (shape) se debe indicar




In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(7, 2.5), tight_layout=True)
for i, data in enumerate([data1, data2, data3]):
    #data = (data - np.mean(data))/np.std(data)
    (osm, osr), (w, b, r2) = scipy.stats.probplot(data, dist="norm", fit=True);
    ax[i].plot(osm, osr)
    ax[i].plot(np.arange(-3, 4), np.arange(-3, 4)*w + b, 'k--', alpha=0.25)
    ax[i].set_title('data'+str(i));

### Nuestro primer test no-paramétrico:  Kolmogorov-Smirnov (KS)

Este test examina la distancia entre las funciones de distribución acumulada (CDF) de dos muestras

- Solo se puede usar para distribuciones univariadas continuas
- Si se compara una muestra empírica con una CDF teórica: Test de KS de una muestra
- Si se comparan dos muestras empíricas: Test de KS de dos muestras

> Test no-paramétrico: No asume distribuciones

El estadístico de test es para KS de una muestra es

$$
D_n = \sup_x |F_n(x) - F(x)|
$$

donde $F(x) = \int_{-\infty}^x f(x) dx = P(X<x)$ es la CDF teórica y 

$$
F_n(x) = \frac{1}{N} \sum_{i=1}^N \mathbb{1}[X_i<x]
$$

es la distribución acumulada empírica (ECDF)

In [ ]:
def ecdf(data):
    x = np.sort(data)
    n = x.size
    y = np.arange(1, n+1) / n
    return(x,y)

fig, ax = plt.subplots(1, 3, figsize=(7, 2.5), tight_layout=True)
for i, data in enumerate([data1, data2, data3]):
    # ECDF
    data = (data - np.mean(data))/np.std(data)
    x, y = ecdf(data)
    ax[i].plot(x, y)
    # CDF standard normal
    x_t = np.linspace(np.amin(data), np.amax(data), num=100)
    y_t = scipy.stats.norm.cdf(x_t)    
    ax[i].plot(x_t, y_t, 'k--', alpha=0.5)

> $D_n$ es la distancia más larga entre todas las distancias de la CDF teórica y empírica

Las hipótesis de la prueba KS de una muestra son

- $\mathcal{H}_0$: Los datos siguen la distribución teórica especificada
- $\mathcal{H}_A$: Los datos no siguen la distribución teórica especificada

Si $D_n$ es menor que el valor crítico no podemos rechazar $H_0$

Podemos usar `scipy.stats.kstest` para probar nuestros datos contra la distribución normal

In [ ]:
for i, data in enumerate([data1, data2, data3]):
    data = (data - np.mean(data))/np.std(data)
    Dn, pvalue = scipy.stats.kstest(data, scipy.stats.norm.cdf)
    print(i, Dn, pvalue)
    if pvalue < 0.05:
        display("Rechazo la hipótesis nula: nuestros datos no siguen la distribución especificada")

Problemas de Kolmogorov-Smirnov

- Solo para distribuciones continuas y univariadas
- Es más sensible en el centro que en las colas de la distribución
- Se debe especificar completamente la distribución teórica

Mejora de KS que le da más ponderación a las colas: [Test de Anderson-Darling](https://www.itl.nist.gov/div898/handbook/prc/section2/prc213.htm), con implementación en [`scipy.stats.anderson`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.anderson.html)


# Actividad 

Use las herramientas que acabamos de estudiar para probar la normalidad de los siguientes datos

In [ ]:
data = np.loadtxt('mistery_data.txt')

Transforme los datos usando logaritmo y repita el procedimiento

¿A que distribución corresponde?

# Inferencia no paramétrica

Inferencia: Estimar parámetros o probar hipótesis sobre una población

En inferencia **no parámetrica** usamos estadísticos cuya distribución **no depende de supuestos sobre la distribución de la población**

- No paramétrico $=$ No asumimos distribución para la población 
- No paramétrico $\neq$ Libre de supuestos (e.g. continuidad, muestras independientes)
- No paramétrico $\neq$ Sin parámetros (e.g. la cantidad de bines de un histograma)

Trade-off: 

- Si sus supuestos se cumplen, los métodos parámetricos son la ópcion de mayor poder estadístico (sensibilidad)
- Los métodos no-parámetricos tienen un poder igual o menor pero no requieren de supuestos